# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [1]:
KEYWORDS = ['python', 'windows']

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
import pandas as pd

In [4]:
req = requests.get('http://habr.com/ru/all/')

In [5]:
soup = BeautifulSoup(req.text, 'html.parser')

In [6]:
posts = soup.find_all('article', class_='post')

In [8]:
links = pd.DataFrame()
for post in posts:
    if not post.parent.attrs.get('id'):
        continue
    post_date = post.find('span', class_='post__time')
    title = post.find('a', class_='post__title_link')
    preview = post.find('div', class_='post__text')
    post_req = requests.get(title.attrs.get('href'))
    post_soup = BeautifulSoup(post_req.text, 'html.parser')
    post_content = post_soup.find('div', id='post-content-body')
    if (any([keyword in title.text.lower() for keyword in KEYWORDS]) | any([keyword in preview.text.lower() for keyword in KEYWORDS]) | any([keyword in post_content.text.lower() for keyword in KEYWORDS])):
        row = pd.DataFrame([{
            'date': post_date.text,
            'title': title.text,
            'link': title.attrs.get('href'),
            'text': post_content.text
        }])
        links = pd.concat([links, row])
links

,date,title,link,text
0,сегодня в 10:01,Однобуквенные языки программирования,https://habr.com/ru/company/macloud/blog/552680/,"\n\n\n\r\nСписок языков программирования, назв..."
0,вчера в 19:49,Применение машинного обучения к кинетике ядерн...,https://habr.com/ru/company/skillfactory/blog/...,Рис. 1. Реактор TRIGA на полной мощности.На Ха...
0,вчера в 18:48,Botfather: универсальный фреймворк для автомат...,https://habr.com/ru/company/itsoft/blog/552010/,"Привет, Хабр!В этом посте я хочу познакомить в..."
0,вчера в 18:40,Нагрузочное тестирование на Gatling — Полное р...,https://habr.com/ru/company/otus/blog/552846/,"Эта статья представляет собой полное, подробно..."


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [9]:
EMAIL = [
    'too.alive@gmail.com',
    '24.alexander.kim@gmail.com'
]

In [19]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
payload = {
    'emailAddresses': EMAIL
}
headers = {
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}

In [20]:
req = requests.post(URL, json=payload, headers=headers)

In [36]:
response = req.json()

In [40]:
df = pd.DataFrame()
for email in response['summary'].keys():
    for breach_id in response['summary'][email]['breaches']:
        breach = response['breaches'][str(breach_id)]
        row = pd.DataFrame([{
            'email': email, 
            'date': breach['publishDate'], 
            'source': breach['site'], 
            'description': breach['description']}])
        df = pd.concat([df, row])
df  

,email,date,source,description
0,too.alive@gmail.com,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,too.alive@gmail.com,2018-06-19T00:00:00Z,storelp.ru,"At an unconfirmed date, StoreLP.ru's database ..."
0,too.alive@gmail.com,2019-10-17T00:00:00Z,zynga.com,"In September 2019, the game developer Zynga wa..."
0,too.alive@gmail.com,2019-05-23T00:00:00Z,livejournal.com,"In 2017, social network LiveJournal's database..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

In [41]:
GROUP = 'playpaintball'
TOKEN = ''

In [42]:
URL = 'https://api.vk.com/method/wall.get'
params = {
    'domain': GROUP,
    'access_token': TOKEN,
    'count': 50,
    'v': '5.21'
}

In [43]:
req = requests.get(URL, params)

In [47]:
import datetime as dt

In [49]:
df = pd.DataFrame()
for item in req.json()['response']['items']:
    row = pd.DataFrame([{'date': dt.datetime.fromtimestamp(item['date']), 'text': item['text']}])
    df = pd.concat([df, row])
df

,date,text
0,2021-04-15 18:21:11,
0,2021-04-14 09:27:43,История одной стычки)
0,2021-04-13 08:00:01,25 апреля 2021. Милсим игра «Штурм-оборона. Бо...
0,2021-04-12 12:10:50,"Результаты четвёртого дня лиги. \nКоманда ""Пен..."
0,2021-04-12 11:30:14,
0,2021-04-09 19:52:46,Раскопали залежи подсумков. Разных. Много. Ком...
0,2021-04-09 15:50:11,В рамках конкурса социальных проектов «Шаг впе...
0,2021-04-07 12:44:13,Здравствуйте друзья пейнтболисты. Кто может з...
0,2021-04-07 12:23:19,Проката на Тактическую лигу 11 апреля больше н...
0,2021-04-07 08:23:48,На «Тактическую лигу» 11 апреля осталось 3 ком...


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
